# ELM Proof of Concept #2 : Topic Modelling

* October 5th, 2018
* Ryan Kazmerik, Data Scientist
* Enterprise Data Science, Encana


## Hypothesis
Topic modelling may reveal word relationships in our news articles, which can be used to extract themes or topics per article and for the entire corpus of news articles.

We will test this hypothesis using two different popular topic models and our corpus of ~10,000 news articles from News-API

### Research
**1. LDA (Latent Dirichlet Allocation)**
* generative probability model
* discovers topics through probability distributions
* produces probability weights per word
<br/>[LDA and the ABC news headlines](https://medium.com/ml2vec/topic-modeling-is-an-unsupervised-learning-approach-to-clustering-documents-to-discover-topics-fdfbf30e27df)<br/><br/>

**2. NMF (Non-negative Matrix Factorization)**
* determanistic probability model
* linear algebreic determination
* produces probability weights per word
<br/>[NMF and the New York Times](https://towardsdatascience.com/topic-modeling-for-the-new-york-times-news-dataset-1f643e15caac)

## Experiments
**We need a training set of articles for this experiment.**

Let's load in ~1200 news headlines from Elastic and focus on articles that mention *oil* or *gas*:

In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

docs = es.search(
    index='articles', 
    doc_type='article',
    q='description:"oil" OR description:"gas"',
    filter_path=['hits.hits'],
    _source_include='description,title',
    sort='_id',
    size=100000
)

articles = []

for i,d in enumerate(docs['hits']['hits']):
    desc = (d["_source"]["description"])
    if(desc):
        articles.append(desc)
    i+=1;
    
print("No. training articles:",len(articles))
print()
print("Sample description:",articles[1])

No. training articles: 8284

Sample description: Shares were mixed in Asia on Monday, with Chinese benchmarks leading decliners. The air strikes on Syria appeared to be having scant impact on trading, and oil prices fell back. Eyes were on Chinese GDP data due on Tuesday. KEEPING SCORE: Japan's Nikkei 225 i…


**The sentences in the articles must be parsed to remove stop words, and split into individual words (tokenization). Then the words need to be encoded as floating point values to be used as input for our algorithms (vectorization).**

Let's create some feature vectors for our dataset:

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text 

n_features = 1000

my_additional_stop_words = ['ap','said','says','time','monday','november']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, analyzer='word',
                         max_features=n_features, stop_words=stop_words, token_pattern = r'\b[a-zA-Z]{3,}\b')

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features, 
                        stop_words=stop_words)

fs1_train = count_vectorizer.fit_transform(articles)
fs2_train = tfidf_vectorizer.fit_transform(articles)

print("Count Feature set shape:",fs1_train.shape)
print("TF-IDF Feature set shape:",fs2_train.shape)

Count Feature set shape: (8284, 1000)
TF-IDF Feature set shape: (8284, 1000)


**Next we will fit our topic models for both algorithms (LDA and NMF). We also need to specify the number of topics we are looking for. We can fine-tune this later but let's start by looking for 6 topics.**

Let's generate our topic models:

In [3]:
from sklearn.decomposition import LatentDirichletAllocation, NMF
from time import time

n_components = 5

print ('Fitting the LDA model...')     
t0 = time()

LDA = LatentDirichletAllocation(n_components=n_components, max_iter=5,
            learning_method='online', learning_offset=50., random_state=0)
LDA = LDA.fit(fs1_train)

print ('   done in:', (time()-t0))
print ()

print ('Fitting the NMF model...')
t0 = time()

NMF = NMF(n_components=n_components, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
NMF = NMF.fit(fs2_train)

print ('   done in:', (time()-t0))

Fitting the LDA model...
   done in: 7.1941375732421875

Fitting the NMF model...
   done in: 0.25736021995544434


## Results
**The most widely used method or evaluating a topic model is extrinsic evaluation, which means we manually evaluate the related words that the topic model generated to see if they make sense.**

Let's have a look at the output of our two topic models:

In [4]:
n_terms = 10
terms1 = count_vectorizer.get_feature_names()
terms2 = tfidf_vectorizer.get_feature_names()

print("LDA - Topic model:")

for topic_idx, topic in enumerate(LDA.components_):
    message = " Topic %d: " % topic_idx
    message += " ".join([terms1[i]
                    for i in topic.argsort()[:-n_terms - 1:-1]])
    print(message)
print()


print("NMF - Topic model:")

for topic_idx, topic in enumerate(NMF.components_):
    message = " Topic %d: " % topic_idx
    message += " ".join([terms2[i]
                    for i in topic.argsort()[:-n_terms - 1:-1]])
    print(message)
print()

LDA - Topic model:
 Topic 0: oil iran sanctions trump president washington iranian exports donald states
 Topic 1: oil energy prices companies stocks stock higher canada index shares
 Topic 2: gas energy oil natural reuters company pipeline new billion liquefied
 Topic 3: oil prices crude opec supply global output production year demand
 Topic 4: oil saudi energy arabia minister state reuters world company agency

NMF - Topic model:
 Topic 0: prices oil crude sanctions supply global iran markets rose demand
 Topic 1: gas energy natural company reuters new oil pipeline billion liquefied
 Topic 2: stock index canada main shares higher energy prices lower stocks
 Topic 3: saudi arabia opec russia oil output minister production aramco al
 Topic 4: trump president donald iran administration sanctions nuclear oil deal washington



## Observations

### 1. Topics are representative of current stories



**The topics generated by NMF seem to be more cohesive and make more sense than the LDA topics. We can see mention of events we know to be happening such as: Iranian sanctions, Hurricane Michael and Russia/Saudi discussing increasing production.**

Let's have a closer look at the NMF topic model and some sample articles for each topic:

In [5]:
import numpy as np

def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d" % (topic_idx))
        print("Keywords:", " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print("Top document:", documents[doc_index])
            
        print()

In [6]:
nmf_W = NMF.transform(fs2_train)
nmf_H = NMF.components_

no_top_words = 10
no_top_documents = 1
display_topics(nmf_H, nmf_W, terms2, articles, no_top_words, no_top_documents)

Topic 0
Keywords: prices oil crude sanctions supply global iran markets rose demand
Top document: Brent crude oil prices rose back above $80 a barrel on Monday as markets were expected to tighten once U.S. sanctions against Iran's crude exports are implemented next month.

Topic 1
Keywords: gas energy natural company reuters new oil pipeline billion liquefied
Top document: Natural gas coalition launches

Topic 2
Keywords: stock index canada main shares higher energy prices lower stocks
Top document: Canada's main stock index opened higher on Monday as a rise in oil prices lifted shares of energy companies.

Topic 3
Keywords: saudi arabia opec russia oil output minister production aramco al
Top document: World’s most profitable company recruits American oil executive Lynn Laverty Elsenhans in milestone move for Saudi Arabia Saudi Aramco, the world’s most profitable company, has appointed a woman to its board, in a milestone move for Saudi Arabia, where only o…

Topic 4
Keywords: trump p

### 2. Number of topics is an important variable

**Too few topics results in really broad general topics.**

In [8]:
import pyLDAvis
import pyLDAvis.sklearn
import warnings; warnings.simplefilter('ignore')
warnings.simplefilter('ignore')

n_components = 3

LDA = LatentDirichletAllocation(n_components=n_components, max_iter=5,
            learning_method='online', learning_offset=50., random_state=0)
LDA = LDA.fit(fs1_train)
    
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA, fs1_train, count_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.139563 -0.023795       1        1  51.906406
2      0.099412 -0.072162       2        1  37.378173
0      0.040151  0.095958       3        1  10.715421, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
401   Default  156.000000            gas  156.000000  30.0000  30.0000
328   Default  138.000000         energy  138.000000  29.0000  29.0000
578   Default   58.000000        natural   58.000000  28.0000  28.0000
390   Default   18.000000       fracking   18.000000  27.0000  27.0000
741   Default   95.000000          saudi   95.000000  26.0000  26.0000
766   Default   19.000000          shale   19.000000  25.0000  25.0000
655   Default  164.000000         prices  164.000000  24.0000  24.0000
995   Default   19.000000          years   19.000000  23.0000  23.0000
228   Default  129.000000          crude  129.000000  22.0000  22.0000
179   Default   36.000000      companies   36.000000  21.0000  21.0000
522   Default   18.000000           long   18.000000  20.0000  20.0000
45    Default   52.000000         arabia   52.000000  19.0000  19.0000
180   Default   38.000000        company   38.000000  18.0000  18.0000
554   Default   36.000000         mexico   36.000000  17.0000  17.0000
230   Default   10.000000      cuadrilla   10.000000  16.0000  16.0000
654   Default   19.000000          price   19.000000  15.0000  15.0000
609   Default   44.000000         output   44.000000  14.0000  14.0000
919   Default   25.000000          trump   25.000000  13.0000  13.0000
490   Default   22.000000     journalist   22.000000  12.0000  12.0000
284   Default   14.000000  disappearance   14.000000  11.0000  11.0000
862   Default   55.000000         supply   55.000000  10.0000  10.0000
608   Default   10.000000        outlook   10.000000   9.0000   9.0000
602   Default  471.000000            oil  471.000000   8.0000   8.0000
131   Default   25.000000         canada   25.000000   7.0000   7.0000
991   Default    8.000000        worries    8.000000   6.0000   6.0000
208   Default   23.000000           corp   23.000000   5.0000   5.0000
891   Default   12.000000           term   12.000000   4.0000   4.0000
268   Default   36.000000         demand   36.000000   3.0000   3.0000
836   Default    7.000000         stoked    7.000000   2.0000   2.0000
371   Default   42.000000           fell   42.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
468    Topic3    2.583051        inherit    3.833602   1.8387  -5.9549
257    Topic3    2.492787       decision    3.740421   1.8277  -5.9905
192    Topic3    2.890660     consortium    4.404025   1.8125  -5.8424
213    Topic3    2.866420         costly    4.395679   1.8059  -5.8508
608    Topic3    5.980886        outlook   10.765257   1.6457  -5.1153
805    Topic3    4.158319           soon    7.128087   1.6946  -5.4787
522    Topic3    8.305295           long   18.437765   1.4360  -4.7870
666    Topic3    4.996077      prominent    9.711414   1.5688  -5.2952
236    Topic3    3.205701      customers    5.354007   1.7206  -5.7389
438    Topic3    3.600281        history    6.468266   1.6476  -5.6228
891    Topic3    5.299031           term   12.207027   1.3990  -5.2363
284    Topic3    5.578447  disappearance   14.238326   1.2965  -5.1849
654    Topic3    6.692555          price   19.816298   1.1480  -5.0029
401    Topic3   19.903748            gas  156.469052   0.1715  -3.9129
602    Topic3   22.050929            oil  471.749565  -0.8296  -3.8105
820    Topic3    3.425482          start    7.025156   1.5152  -5.6726
862    Topic3    7.864455         supply   55.494249   0.2796  -4.8415
490    Topic3    5.360364     journalist   22.336772   0.8063  -5.2248
537    Topic3    2.888853          maine    4.803700   1.7250  -5.

In [9]:
n_components = 20

LDA = LatentDirichletAllocation(n_components=n_components, max_iter=5,
            learning_method='online', learning_offset=50., random_state=0)
LDA = LDA.fit(fs1_train)
    
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA, fs1_train, count_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16    -0.130020 -0.034879       1        1  42.655558
9     -0.205483 -0.004641       2        1  17.632574
13     0.033393 -0.179493       3        1  15.524395
10     0.092094 -0.042597       4        1  10.300008
3      0.005810 -0.003625       5        1   2.176049
7      0.043386  0.035337       6        1   2.069388
8      0.015946  0.021994       7        1   1.466527
4      0.008884  0.020584       8        1   1.166162
1      0.022794  0.025394       9        1   1.145703
18     0.010098  0.026640      10        1   0.920353
15     0.017738  0.010999      11        1   0.759390
17     0.018057  0.017938      12        1   0.632867
0      0.012227  0.024030      13        1   0.615180
19     0.015677  0.011075      14        1   0.575792
12     0.015512  0.012684      15        1   0.535506
5      0.006042  0.015707      16        1   0.535338
11     0.001046  0.011390      17        1   0.375470
6      0.001995  0.009265      18        1   0.352021
2     -0.001258  0.007544      19        1   0.280859
14     0.016062  0.014654      20        1   0.280859, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
602   Default  434.000000             oil  434.000000  30.0000  30.0000
401   Default  147.000000             gas  147.000000  29.0000  29.0000
328   Default  129.000000          energy  129.000000  28.0000  28.0000
655   Default  156.000000          prices  156.000000  27.0000  27.0000
228   Default  120.000000           crude  120.000000  26.0000  26.0000
741   Default   88.000000           saudi   88.000000  25.0000  25.0000
578   Default   54.000000         natural   54.000000  24.0000  24.0000
543   Default   48.000000         markets   48.000000  23.0000  23.0000
862   Default   52.000000          supply   52.000000  22.0000  22.0000
180   Default   35.000000         company   35.000000  21.0000  21.0000
473   Default   44.000000     inventories   44.000000  20.0000  20.0000
45    Default   48.000000          arabia   48.000000  19.0000  19.0000
994   Default   51.000000            year   51.000000  18.0000  18.0000
179   Default   34.000000       companies   34.000000  17.0000  17.0000
920   Default   31.000000         tuesday   31.000000  16.0000  16.0000
348   Default   34.000000        expected   34.000000  15.0000  15.0000
609   Default   40.000000          output   40.000000  14.0000  14.0000
833   Default   33.000000           stock   33.000000  13.0000  13.0000
989   Default   44.000000           world   44.000000  12.0000  12.0000
973   Default   47.000000            week   47.000000  11.0000  11.0000
411   Default   55.000000          global   55.000000  10.0000  10.0000
481   Default   52.000000            iran   52.000000   9.0000   9.0000
554   Default   34.000000          mexico   34.000000   8.0000   8.0000
584   Default   26.000000             new   26.000000   7.0000   7.0000
354   Default   26.000000         exports   26.000000   6.0000   6.0000
919   Default   24.000000           trump   24.000000   5.0000   5.0000
424   Default   34.000000            gulf   34.000000   4.0000   4.0000
72    Default   23.000000          barrel   23.000000   3.0000   3.0000
371   Default   41.000000            fell   41.000000   2.0000   2.0000
268   Default   36.000000          demand   36.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
183   Topic20    0.029225      complaints    3.380689   1.1243  -6.7950
26    Topic20    0.030148          ailing    3.629010   1.0845  -6.7639
792   Topic20    0.030046         sinking    3.698166   1.0622  -6.7673
419   Topic20    0.028381        gripping    3.174807   1.1578  -6.8243
8     Topic20    0.028787           adani    3.322950   1.1264  -6.8101
774   Topic20    0.029724        

## Conclusion

*Hypothesis: Topic modelling may reveal word relationships in our news articles, which can be used to extract themes or topics per article and for the entire corpus of news articles.*

* Non-negative matrix factorization does a better job of clustering related words given a relatively small dataset (~1200 headlines), where as LDA struggled to provide cohesive results at this scale.
<br/>

* Each article could be classified by our topic model as we process new articles through the pipeline efficiently. This would provide a high-level (macro) category, but more granular tagging would require a different approach.
<br/>

* Difficult to determine the exact accuracy of the topic model, as the primary means of evaluation is extrinsic (human evaluation).

## Further Improvements

1. Changing the initial query to Elastic had a significant impact on the topics produced. We could experiment with this query to dial in how broad or specific the topics are.
<br/>

2. The LDA model did not perform well with ~1200 articles but may perform better with a larger dataset - as generative models typically need lots of data.
<br/>

3. We may consider writing an additional component that labels the topics programatically. Example: query another source with the top 10 keywords and receive a high level topic label.
<br/>

4. Could add many more terms to the stop-words list to prevent topics from containing generic words such as 'tuesday'.
<br/>

5. Need to experiment with bi-grams and tri-gram vectorizers so that keyword phrases like 'british columbia' do not appear as seperate keywords 'british' and 'columbia'. 

6. Should implement stemming, lemmatization, PoS tagging.